# Day 1 - Points

Data sources:
- [Station coordinates](https://www.rijdendetreinen.nl/en/open-data/stations#downloads)
- [Train prognoses](https://www.rijksoverheid.nl/documenten/rapporten/2021/06/29/bijlage-3-achtergrondrapport-2-spoor-en-btm)

In [51]:
from IPython.display import display
import folium
import pandas as pd
import numpy as np

In [52]:
# Variables
metric = "2050 Hoog"
stations_path = "data/stations-2023-09.csv"
ima_path = "data/ima/ima_2021_train_travellers.csv"

In [53]:
# Load data
stations = pd.read_csv(stations_path)
ima = pd.read_csv(ima_path)

In [54]:
# Fix station names in ima data so it matches with the stations data
mapping = {
    "Alphen aan den Rijn":"Alphen a/d Rijn",
    "Amsterdam Sciencepark":"Amsterdam Science Park",
    "Breda Prinsenbeek":"Breda-Prinsenbeek",
    "Den Haag Laan van NOI":"Den Haag Laan v NOI",
    "Doetinchem de Huet":"Doetinchem De Huet",
    "Harde ‘t":"'t Harde",
    "Helmond ‘t Hout":"Helmond 't Hout",
    "Hertogenbosch ‘s":"'s-Hertogenbosch",
    "Hertogenbosch ‘s Oost":"'s-Hertogenbosch Oost",
    "Houthem-Sint Gerlach":"Houthem-St. Gerlach",
    "Leeuwarden Werpsterhoeke":"Leeuwarden-Werpsterhoeke",
    "Mook Molenhoek":"Mook-Molenhoek",
    "Nieuwerkerk aan den IJssel":"Nieuwerkerk a/d IJssel",
    "Stadskanaal":"",
    "Veenendaal-de Klomp":"Veenendaal-De Klomp",
}
ima["Stationsnaam"] = ima["Stationsnaam"].replace(mapping)

In [55]:
# Merge station and ima data
merged_df = stations.merge(ima, left_on="name_long", right_on="Stationsnaam", how="left")

merged_df.head(3)

,id,code,uic,name_short,name_medium,name_long,slug,country,type,geo_lat,geo_lng,Stationsnaam,2030 Laag,2040 Laag,2050 Laag,2030 Hoog,2040 Hoog,2050 Hoog
0,266,HT,8400319,Den Bosch,'s-Hertogenbosch,'s-Hertogenbosch,s-hertogenbosch,NL,knooppuntIntercitystation,51.690480,5.293620,'s-Hertogenbosch,100.0,100.0,105.0,103.0,112.0,123.0
1,269,HTO,8400320,Dn Bosch O,'s-Hertogenb. O.,'s-Hertogenbosch Oost,s-hertogenbosch-oost,NL,stoptreinstation,51.700554,5.318333,'s-Hertogenbosch Oost,88.0,89.0,93.0,90.0,98.0,105.0
2,227,HDE,8400388,'t Harde,'t Harde,'t Harde,t-harde,NL,stoptreinstation,52.409168,5.893611,'t Harde,105.0,108.0,115.0,106.0,117.0,130.0


In [56]:
# Add column to data with color for plotting
def determine_color(x):
    if x > 120:
        return "#9fd55c"
    elif x > 105:
        return "#5fb57e"
    elif x > 95:
        return "#448AFF"
    elif x > 80:
        return "#F44336"
    elif x <= 80:
        return "#AD1B10"
    else:
        return "#607D8B"

merged_df["color"] = merged_df[metric].apply(lambda x: determine_color(x))

In [57]:
# Map settings
map_center = [52, 5.5]
zoom_start = 7

# Define map
m = folium.Map(location=map_center, zoom_start=zoom_start, tiles="cartodbpositron")

# Add title
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Expected growth of train stations in 2040</b></h3>
             <p align="center">
               <span style="color:#4CAF50">increase in passengers</span> - 
               <span style="color:#F44336;">decrease in passengers</span> -  
               <span style="color:#448AFF;">no change expected</span> -   
               <span style="color:#607D8B;">unknown</span>
            </p>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Add markers for each second hand store
for index, row in merged_df.iterrows():
    # Determine size
    if np.isnan(row[metric]):
        size = 4
        tooltip = f"{row['name_long']} ({row['country']})"
    else: 
        size = 4 * row[metric] / 100
        tooltip = f"{row['name_long']} ({row['country']}): {row[metric]-100:.0f}%"

    # Create the marker
    folium.CircleMarker(
        location=[row["geo_lat"], row["geo_lng"]],
        radius=size,
        color=None,
        fill=True,
        fill_color=row["color"],
        fill_opacity=0.5,
        tooltip=tooltip
    ).add_to(m)

# Display the map
display(m)

In [58]:
m.save("../maps/day1_points.html")